## IMPORT DATA

In [5]:
import pandas as pd
import numpy as np

In [6]:
# Cargamos los datos de entrenamiento y test desde archivos CSV. Como los datasets son muy grandes y tardan en cargarse, utilizamos una librería para paralelizar el código de carga.
# Librería utilizada: multiprocessing
from multiprocessing.pool import ThreadPool

# Función para cargar los archivos csv que tenemos como datasets.
def carregar(nom):
    return pd.read_csv(nom)

# Los csv que cargaremos son los siguientes:
csvs = ['fraudTrain.csv', 'fraudTest.csv']

# Creamos 2 grupos porque tenemoso dos archivos csv.
pool = ThreadPool(2)

# Cargamos los archivos en paralelo.
resultats = pool.map(carregar, csvs)

# Guardamos los resultados en sus respectivas variables de dataset de entrenamiento y test.
train_data = resultats[0]
test_data = resultats[1]

In [7]:
print(f'{train_data.shape}')
print(f'{test_data.shape}')

(1296675, 23)
(555719, 23)


### Transform data to timestamp

In [8]:
train_data = train_data.drop(columns='Unnamed: 0')
test_data = test_data.drop(columns='Unnamed: 0')

In [9]:
# Ahora, convertimos las fechas a números, específicamente a timestamps en segundos desde la época Unix (1 de enero de 1970).
train_data['trans_date_trans_time'] = pd.to_datetime(train_data['trans_date_trans_time']).astype('int64') // 10**9
test_data['trans_date_trans_time'] = pd.to_datetime(test_data['trans_date_trans_time']).astype('int64') // 10**9

# Convertimos la columna 'dob' (fecha de nacimiento) a timestamps en segundos.
train_data['dob'] = pd.to_datetime(train_data['dob']).astype('int64') // 10**9
test_data['dob'] = pd.to_datetime(test_data['dob']).astype('int64') // 10**9

In [10]:
test_data.head(1)

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,1592741665,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,...,33.9659,-80.9355,333497,Mechanical engineer,-56419200,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0


#### CODIFICAMOS LOS DATOS CON UN MODELO HÍBRIDO (ONE-HOT ENCODER Y ORDINAL ENCODER)

In [11]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

# Eliminamos las columnas de IDs que no aportan valor a la predicción y pueden causar sobreajuste. Tanto como para los datos de entrenamiento como de test.
train_data = train_data.drop(columns=['cc_num', 'trans_num'])
test_data = test_data.drop(columns=['cc_num', 'trans_num'])

# Columnas buenas candidatas para OneHotEncoder (pocos valores únicos):
low_cardinality_cols = ['gender', 'category'] 

# Columnas buenas candidatas para OrdinalEncoder (muchos valores únicos):
high_cardinality_cols = ['merchant', 'job', 'city', 'state', 'street', 'first', 'last']

# Aquí aplicamos OneHotEncoder a las columnas ya definidas.
encoder_onehot = OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype=int) # Con el handle_unknown='ignore' evitamos errores si en test hay categorías no vistas en train.

# Ahora aplicamos OrdinalEncoder a las columnas ya definidas.
encoder_ordinal = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, dtype=int) # handle_unknown='use_encoded_value' con unknown_value=-1 asigna -1 a categorías no vistas en test.

# Entrenamos los codificadores con los datos de entrenamiento (fit).
encoder_onehot.fit(train_data[low_cardinality_cols])
encoder_ordinal.fit(train_data[high_cardinality_cols])

# Función para aplicar las transformaciones en paralelo.
def aplicar_encoders(df):
    # Transformamos los datos (ajustados previamente con el train).
    onehot_out = encoder_onehot.transform(df[low_cardinality_cols])
    ordinal_out = encoder_ordinal.transform(df[high_cardinality_cols])
    
    # Convertimos los arrays resultantes a DataFrames para facilitar su manejo (para concatenar, etc.).
    onehot_df = pd.DataFrame(onehot_out, index=df.index)
    ordinal_df = pd.DataFrame(ordinal_out, index=df.index)
    
    # Eliminamos las columnas categóricas originales de ambos conjuntos de datos para luego añadir las codificadas.
    df_temp = df.drop(columns=low_cardinality_cols + high_cardinality_cols)
    
    # Ahora concatenamos las nuevas columnas codificadas a los DataFrames originales. Ya sin las columnas categóricas originales.
    return pd.concat([df_temp, onehot_df, ordinal_df], axis=1)

# Creamos un grupo de 2 procesos para procesar train y test simultáneamente.
pool_enc = ThreadPool(2)
resultats_enc = pool_enc.map(aplicar_encoders, [train_data, test_data])

# Guardamos los resultados en las correspondientes variables de train y test.
train_data = resultats_enc[0]
test_data = resultats_enc[1]

## Entrenamos el Autoencoder

In [ ]:
# Importamos las librerías de PyTorch, sklearn y matplotlib necesarias para construir, entrenar y evaluar el autoencoder.
# Las librerías en concreto son, explicadas brevemente:
# - torch: La librería principal de PyTorch para construir nuestros modelos de ML.
# - torch.nn: Módulo de PyTorch que contiene clases y funciones para construir redes neuronales.
# - torch.optim: Propociona algoritmos de optimización para entrenar modelos.
# - DataLoader y TensorDataset: Para manejar conjuntos de datos y cargarlos en mini-batches.
# - RobustScaler: Escalador de scikit-leaern muy útil para datos con outliers.
# - confusion_matrix, classification_report, recall_score, precision_score, f1_score, roc_auc_score: Métricas de evaluación para visualizar los resultados.
# - matplotlib.pyplot: Librería para crear gráficos.    
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import confusion_matrix, classification_report, recall_score, precision_score, f1_score, roc_auc_score


# Separamos las features y target:
X_train_np = train_data.drop(columns='is_fraud').values # values transforma de un pandas DataFrame a un numpy array.
y_train_np = train_data['is_fraud'].values
X_test_np = test_data.drop(columns='is_fraud').values
y_test_np = test_data['is_fraud'].values


# Cogemos solo las transacciones no fraudulentas para entrenar el autoencoder, que se encargará de aprender el patrón de las transacciones normales y luego detectar anomalías, que serían las fraudulentas.
train_index_normal = np.where(y_train_np == 0)[0] 
X_train_normal = X_train_np[train_index_normal] # X_train_normal contiene solo las transacciones no fraudulentas.
# Separamos los datos normales (no fraudulentos) para el entrenamiento, pero para el test usamos todos ellos, para las predicciones y que el modelo detecte anomalías como fraudes.


# RobustScaler utiliza la mediana y el IQR, mejor para datos con outliers extremos (escalamos por columnas, no por filas y por tanto no afecta datos binarios como las one-hot encoded).
# Como el dataset es muy grande, con millones de filas, vamos a paralelizar también el escalado porque suele ser un proceso lento y costoso.
scaler = RobustScaler()
scaler.fit(X_train_normal)

# Función para aplicar la transformación del escalador en paralelo.
def aplicar_escalado(datos):
    return scaler.transform(datos)

# Creamos un grupo de 2 hilos para transformar los datos de entrenamiento y test simultáneamente.
pool_scaler = ThreadPool(2)
resultados_escalado = pool_scaler.map(aplicar_escalado, [X_train_normal, X_test_np])

# X_train_scaled contiene los datos normales escalados.
X_train_scaled = resultados_escalado[0]
# X_test_scaled contiene todos los datos de test escalados.
X_test_scaled = resultados_escalado[1]

#CONVERSIÓN A PYTORCH:
train_tensor = torch.FloatTensor(X_train_scaled) 
test_tensor = torch.FloatTensor(X_test_scaled)

batch_size = 256  # Más pequeño para mejor convergencia y menos memoria. Con batchs grandes puede que el modelo no aprenda bien, y con más pequeños el gradiente es más grande y permite mejores actualizaciones.
# Train loader lo que hace es crear mini-batches de datos para entrenar el modelo de forma más eficiente. Es específico de PyTorch.
# Para paralelizar esta parte del código, utilizamos num_workers=4 para cargar los datos en paralelo. Creamos 4 procesos que trabajan en paralelo.
train_loader = DataLoader(TensorDataset(train_tensor), batch_size=batch_size, shuffle=True, num_workers=4) # quitamos el num_workers para que se ejecute en secuencial.



# Ahora definimos el autoencoder con PyTorch. Esta clase define la arquitectura del modelo con las capas del encoder y decoder.

class DeepFraudAutoencoder(nn.Module): # Heredamos de nn.Module, la clase base para todos los modelos de PyTorch.
    
    # Arquitectura del autoencoder con un encoder y decoder más profundos y con dropout para evitar overfitting.
    # Aquí el modelo lo que hace es descomponer los datos de entrada en una representación comprimida (encoder) y luego reconstruirlos (decoder). Luego, al comparar la entrada original
    # con la reconstruida, podemos detectar anomalías (fraudes) basándonos en el error de reconstrucción, ya que las transacciones fraudulentas deberían tener un error mayor al no seguir 
    # el patrón aprendido de las transacciones normales y presentar mayores anomalías.
    
    def __init__(self, input_size):
        super(DeepFraudAutoencoder, self).__init__() # Super llama al constructor de la clase base nn.Module.
        
        # Encoder más profundo con dropout y ReLU como función de activación, que ayuda a capturar patrones complejos en los datos.
        # Pasamos del número de columnas de entrada a una representación comprimida de 64, 48, 32, 16 y finalmente 8, el bottleneck.
        # El dropout ayuda a prevenir el sobreajuste durante el entrenamiento. Lo que hace es desactivar aleatoriamente un porcentaje de neuronas en cada capa durante el entrenamiento, para 
        # evitar que el modelo dependa demasiado de ciertas neuronas y así mejorar su capacidad de aprendizaje general.
        # El linear lo que hace es aplicar una transformación lineal a los datos de entrada, es decir, multiplica los datos por una matriz de pesos y añade un sesgo. Esto permite 
        # al modelo aprender relaciones lineales entre las características de entrada.
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(64, 48),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(48, 40),
            nn.ReLU(),
            
            nn.Linear(40, 32)  # Cuello de botella (bottleneck), es el final del encoder. Aquí los datos están más comprimidos y se supone que el modelo ha aprendido las características más importantes.
        )
        
        # Decoder simètrico al encoder, pero en sentido contrario, para reconstruir los datos originales a partir de la representación comprimida. Es decir, a partir de un cuello de botella 
        # de 8 dimensiones, vamos expandiendo de nuevo a 16, 32, 48, 64 y finalmente al número de columnas originales. En esta reconstrución, el modelo intentará reconstruir los datos
        # de entrada lo mejor posible. No obstante, cometerá errores, sobretodo en las transacciones fraudulentas porque esas no las ha visto durante el entrenamiento. Luego, al calcular el 
        # error cometido en la reconstrucción, teóricamente los mayores errores se daran en fraudes.
        self.decoder = nn.Sequential(
            nn.Linear(32, 40),
            nn.ReLU(),
            
            nn.Linear(40, 48),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(48, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(128, input_size)
        )

    # Definimos la función forward, que define cómo los datos fluyen a través del modelo. Esto lo que hace es pasar los datos de entrada por el encoder y luego por el decoder para obtener la reconstrucción.
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


# Ahora creamos una instancia del modelo. Definimos la dimensión de entrada que deberá ser exactamente igual al número de columnas de los datos de entrada, las features.
input_dim = X_train_scaled.shape[1]
model = DeepFraudAutoencoder(input_dim)

# El criterio de pérdida que utilizaremos será el MSE (Mean Squared Error), que mide la diferencia cuadrática media entre los valores originales y los reconstruidos.
# El optimizador utilizado es el AdamW, una variante de Adam que incluye decaimiento de peso, útil para evitar overfitting. 
# También definimos un scheduler que reduce la tasa de aprendizaje si la pérdida no mejora durante varias épocas.
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)

# Luego, definimos el bucle de entrenamiento: especificamos el número de épocas, inicializamos variables para el early stopping y almacenamos las pérdidas de entrenamiento.
EPOCHS = 100
best_loss = float('inf') # Inicializamos la mejor pérdida con infinito, para que cualquier pérdida calculada sea mejor al principio.
notepoch = 15 # Con esto, decimos que pare el entrenamiento si no mejora la pérdida en 15 épocas consecutivas.
notepoch_counter = 0 # Esto es el contador de épocas sin mejora
train_losses = [] # Guardamos los errores de entrenamiento.
best_model_weights = None  # Guardar el mejor modelo en memoria

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    for data in train_loader:
        inputs = data[0]
        
        # Forward para obtener las salidas y calcular la pérdida. Esta parte lo que hace es pasar los datos de entrada por el modelo para obtener las reconstrucciones y 
        # luego calcular la pérdida entre las entradas originales y las reconstruidas.
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        
        # Backward para actualizar los pesos. Esta parte lo que hace es calcular los gradientes de la pérdida con respecto a los pesos del modelo y luego actualizar 
        # esos pesos utilizando el optimizador.
        optimizer.zero_grad()
        loss.backward()
        
        # Clipping de los gradientes para evitar que se vuelvan demasiado grandes, lo que puede causar inestabilidad en el entrenamiento. Esto se trata de 
        # limitar la norma de los gradientes a un valor máximo. La norma máxima impuesta es 1.0.
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step() # Este paso actualiza los pesos del modelo basándose en los gradientes calculados.
        
        train_loss += loss.item() # Acumulamos la pérdida de entrenamiento de este batch, para luego calcular la pérdida media al final de la época.
    
    # Una vez hemos pasado por todos los batches, calculamos la pérdida media de la época y la almacenamos.
    avg_loss = train_loss / len(train_loader)
    train_losses.append(avg_loss) # Lo guardamos para diferentes épocas.
    
    # Para ver la época cuando ejecutamos el código.
    print(f"Època [{epoch+1}/{EPOCHS}]")    
    
    
    # Para controlar el early stopping, comprobamos si la pérdida ha mejorado. Si no mejora durante ciertas épocas consecutivas, paramos el entrenamiento.
    
    if avg_loss < best_loss:
        # Aquí, si el error es menor que la best_loss, significa que hemos mejorado. Entonces, best_loss pasa a ser el avg_loss actual, y reseteamos el contador de épocas sin mejora.
        best_loss = avg_loss
        notepoch_counter = 0
        best_model_weights = model.state_dict().copy()  # Guardar pesos trobats
    else:
        # Aquí, si no hemos mejorado, incrementamos el contador de épocas.
        notepoch_counter += 1
        if notepoch_counter >= notepoch:
            # Si llegamos al límite de número de épocas máximo sin mejora, aturamos el entrenamiento.
            print(f"Early stopping a l'època {epoch+1}")
            break
    
    scheduler.step(avg_loss) # Actualizamos el scheduler con la pérdida media de la época. Esto ajustará la tasa de aprendizaje si es necesario.
    


# Cogemos los pesos del mejor modelo guardados durante el entrenamiento y los cargamos en el modelo actual.
model.load_state_dict(best_model_weights)
print(f"\nMillor loss aconseguit: {best_loss:.6f}")


# Ahora, evaluamos el modelo en los datos de test para detectar fraudes. Calculamos las reconstrucciones y luego el error de reconstrucción para cada transacción. Después,
# miramos si el error supera un threshold impuesto y, si lo supera, la transacción se clasifica como fraude, si no como normal.

# Evaluación del modelo y cálculo de errores:
model.eval() 
with torch.no_grad():
    # Ahora hacemos las reconstrucciones de los datos de test. El input son todas las transacciones de test, tanto normales como fraudulentas, y el output son las reconstrucciones.
    reconstructions = model(test_tensor)
    # Calculalmos los errores de reconstrucción entre los datos originales y las reconstrucciones.
    mse_loss = torch.mean((test_tensor - reconstructions) ** 2, dim=1).cpu().numpy()

# Guardamos el error en un diccionario, por si quisieramos también calcular más errores come el MAE, etc.
error_df = pd.DataFrame({
    'mse': mse_loss,
    'true_class': y_test_np
})

# Calculamos el normal_mse y fraud_mse para ver el error en cada clase.
normal_mse = error_df[error_df['true_class'] == 0]['mse']
fraud_mse = error_df[error_df['true_class'] == 1]['mse']


print("DIFERENTS THRESHOLD")

# Probamos distintos valores de threshold basados en valores típicos como percentiles del error de las transacciones normales. Entonces, podemos ver que transacciones estan por encmia
# de estos errores y marcarlos como fraudes.
strategies = {
    'P95': np.percentile(normal_mse, 95), # Error bajo el cual se encuentran el 95% de las transacciones normales.
    'P99': np.percentile(normal_mse, 99), # Error bajo el cual se encuentran el 99% de las transacciones normales.
    'P90': np.percentile(normal_mse, 90), # Error bajo el cual se encuentran el 90% de las transacciones normales.
}


# Evaluamos cada estrategia de threshold y calculamos las métricas de evaluación. Muy importante fijarnos en los valores de los TP, FP y FN.
results = []
for name, threshold in strategies.items():
    # Cogemos el error de las predicciones y lo comparamos con el threshold establecido para clasificar como fraude o transacción normal.
    # Luego y_pred es una lista donde 1 representa fraude y 0 normal.
    y_pred = (mse_loss > threshold).astype(int)
    
    # Calculamos la matriz de confusión y las métricas de recall, precision y f1-score.
    tn, fp, fn, tp = confusion_matrix(y_test_np, y_pred).ravel()
    recall = recall_score(y_test_np, y_pred)
    precision = precision_score(y_test_np, y_pred)
    f1 = f1_score(y_test_np, y_pred)
    
    # Finalmente guardamos los resultados en un diccionario para poder acceder facilmente a ellos después y compararlos.
    results.append({
        'Strategy': name,
        'Threshold': threshold,
        'TP': tp,
        'FP': fp,
        'FN': fn,
        'TN': tn,
        'Recall': recall,
        'Precision': precision,
        'F1': f1
    })
    
    print(f"{name:15s} (T={threshold:.4f}): Recall={recall:.3f}, Prec={precision:.4f}, F1={f1:.4f}, TP={tp}, FP={fp}")


# Transformamos el diccionario de resultados a un DataFrame para facilitar el análisis y cálculo de métricas, tal y como veremos a continuación.
results_df = pd.DataFrame(results)

# Ahora, ponemos un peso mayor al recall en el cálculo del F1-score, ya que en detección de fraudes es más importante detectar la mayoría de fraudes posibles (recall alto),
# aunque eso implique tener más falsos positivos (precision más baja). Por eso, damos un peso del 60% al recall y 40% a la precision.
results_df['F1_Weighted'] = 2 * (0.6 * results_df['Recall'] * results_df['Precision']) / (0.6 * results_df['Recall'] + results_df['Precision'])
# Para hallar el mejor resultado, buscamos cuál tiene el F1_Weighted mayor, ya que será el que mejor balancea recall y precision según las condiciones impuestas.
best_result = results_df.loc[results_df['F1_Weighted'].idxmax()]

print(f"MILLOR ESTRATÈGIA: {best_result['Strategy']}")
print(f"Threshold: {best_result['Threshold']:.6f}")

# Ahora, aplicamos el mejor threshold encontrado para hacer las predicciones finales.
best_threshold = best_result['Threshold']
y_pred_best = (mse_loss > best_threshold).astype(int)


# Ahora vamos a mostrar la matriz de confusión del mejor modelo para ver cómo ha clasificado las transacciones.
print("\nMATRIU DE CONFUSIÓ")
cm = confusion_matrix(y_test_np, y_pred_best)
# Más específicamente, en TN, FP, FN, TP:
tn, fp, fn, tp = cm.ravel()
print(f"\nTrue Negatives (TN):  {tn:6d} - Normals correctament classificades")
print(f"False Positives (FP): {fp:6d} - Normals marcades com a frau")
print(f"False Negatives (FN): {fn:6d} - Fraus no detectats")
print(f"True Positives (TP):  {tp:6d} - Fraus detectats correctament ✓")


# Mostramos la tabla final de resultados:
print("\nTAULA COMPARATIVA D'ESTRATÈGIES")
print(results_df.to_string(index=False))

Època [1/100]
Època [2/100]
Època [3/100]
Època [4/100]
Època [5/100]
Època [6/100]
Època [7/100]
Època [8/100]
Època [9/100]
Època [10/100]
Època [11/100]
Època [12/100]
Època [13/100]
Època [14/100]
Època [15/100]
Època [16/100]
Època [17/100]
Època [18/100]
Època [19/100]
Època [20/100]
Època [21/100]
Època [22/100]
Època [23/100]
Època [24/100]
Època [25/100]
Època [26/100]
Època [27/100]
Època [28/100]
Època [29/100]
Època [30/100]
Època [31/100]
Època [32/100]
Època [33/100]
Època [34/100]
Època [35/100]
Època [36/100]
Època [37/100]
Època [38/100]
Època [39/100]
Època [40/100]
Època [41/100]
Època [42/100]
Època [43/100]
Època [44/100]
Època [45/100]
Època [46/100]
Època [47/100]
Època [48/100]
Època [49/100]
Època [50/100]
Època [51/100]
Època [52/100]
Època [53/100]
Època [54/100]
Època [55/100]
Època [56/100]
Època [57/100]
Època [58/100]
Època [59/100]
Època [60/100]
Època [61/100]
Època [62/100]
Època [63/100]
Època [64/100]
Època [65/100]
Època [66/100]
Època [67/100]
Èpoc

TAULA COMPARATIVA D'ESTRATÈGIES (resultat amb el bottleneck comentat i mse)
Strategy  Threshold   TP    FP   FN   Recall  Precision       F1  F1_Weighted
     P95   2.330554   97 27679 2048 0.045221   0.003492 0.006484     0.006188
     P99  26.827852   11  5536 2134 0.005128   0.001983 0.002860     0.002412
     P90   0.350100 1121 55358 1024 0.522611   0.019848 0.038244     0.037333

F1-Score:                  0.0382 - Mitjana harmònica
Accuracy:                  0.8985
False Positive Rate (FPR): 0.1000 - % normals marcades com frau
AUC-ROC:                   0.7896